<a href="https://colab.research.google.com/github/Yen-Nhi-23/TTNT/blob/main/Thuat_toan_minmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thuật toán min_max

In [2]:
import copy
import math
import random
import numpy

X = "X"
O = "O"
EMPTY = None
user = None
ai = None

# khoi tao ban co
def initial_state():
  return [[EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY]]


def player(board):
  count = 0
  for i in board:
    for j in i:
      if j:
        count += 1
  if count % 2 != 0:
    return ai
  return user

def get_horizontal_winner():
  winner_val = None
  board_len = len(board)
  for i in range(board_len):
    winner_val = board[i][0]
    for j in ran

def winner(board):
  winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
  return winner_val

def terminal(board):
  if winner(board) != None:
    return True
  for i in board:
    for j in i:
      if j == EMPTY:
        return False
  return True


if __name__ == "__main__":
  board = initial_state()
  ai_turn = False
  print("Chọn người chơi")
  user = input()
  if user == "X":
    ai = 'O'
  else:
    ai = "X"

  while True:
    game_over = terminal(board)


SyntaxError: incomplete input (ipython-input-3594953187.py, line 20)